# KE5006 Applied Research

### Identifying enhancers and their strength with deep neural networks

# Data Pre-processing 2

## Summary of Findings
* Make use of the di-nuleotide physiochemical properties as input features
	* for each input sequence, create a sequence of di-nucleotides e.g. ACAC... becomes AC-CA-AC-...
	* convert each sequence of di-nucleotides to its physiochemical property
	* pad each sequence of di-nucleotides physiochemical property with a last value of -1 to get the same length as the input sequence
	* a total of 6 di-nucleotides physiochemical properties are used as features (rise, roll, shift, slides, tilt, twist)
* Combine the new features with the one-hot-encoded nucleotide base features (A, C, T, G)


## Load libraries

In [1]:
# Set the working directory (which contains the directories source, data, etc.)
import os
os.chdir(os.path.join(os.path.sep, 'home', 'tkokkeng', 'Documents', 'KE5006-AppliedResearch', 'enhancer'))
os.getcwd()

'/home/tkokkeng/Documents/KE5006-AppliedResearch/enhancer'

In [2]:
# Check if the directory containing the source files are in the path.
import sys
if os.path.join(os.getcwd(), 'source') not in sys.path:
    sys.path.append(os.path.join(os.getcwd(), 'source'))
sys.path

['/home/tkokkeng/Documents/KE5006-AppliedResearch/enhancer',
 '',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python3.6/site-packages',
 '/home/tkokkeng/Documents/KE5006-AppliedResearch/enhancer/source',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python36.zip',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python3.6',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python3.6/lib-dynload',
 '/usr/lib/python3.6',
 '/home/tkokkeng/.local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python3.6/site-packages/IPython/extensions',
 '/home/tkokkeng/.ipython']

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


## Load data

In [4]:
enhancer_df = pd.read_csv(os.path.join('data', 'enhancer.csv'))
enhancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 2 columns):
id          1484 non-null object
sequence    1484 non-null object
dtypes: object(2)
memory usage: 23.3+ KB


In [5]:
enhancer_df['enhancer'] = np.ones((len(enhancer_df),))

In [6]:
enhancer_df.head()

,id,sequence,enhancer
0,CHRX_48897056_48897256,CACAATGTAGAAGCAGAGACACAGGAACCAGGCTTGGTGATGGCTC...,1.0
1,CHR12_6444339_6444539,GCCCTCACATTCCCTGGCCCATCCCCTCCACCTCAAAATTTACAAA...,1.0
2,CHR12_6444939_6445139,GAGCAGGAGGCCAGTCACCCTGAGTCAGCCACGGGGAGACGCTGCA...,1.0
3,CHR12_6445139_6445339,CCTCTGCTGAGAACAGGACTGGGGCTTCCAGGGCAACAGGAAGGGT...,1.0
4,CHR12_6445339_6445539,ACAGCCTTAAAGGGAGCTTTTCAGGGACCTCTGGCCAGTGGGGGAT...,1.0


In [7]:
non_enhancer_df = pd.read_csv(os.path.join('data', 'non_enhancer.csv'))
non_enhancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 2 columns):
id          1484 non-null object
sequence    1484 non-null object
dtypes: object(2)
memory usage: 23.3+ KB


In [8]:
non_enhancer_df['enhancer'] = np.zeros((len(non_enhancer_df),))

In [9]:
non_enhancer_df.head()

,id,sequence,enhancer
0,CHRX_2970600_2970800,CAGTCACATCTGTAATCACAATACGTTGGGAGGCTGAGGCAGGAGG...,0.0
1,CHRX_6179400_6179600,ACTTTGAAGAAGTCAGTCATCAAGATGAGAGACCCAACTGTCAAGC...,0.0
2,CHRX_11003079_11003279,TCGGCCTCCCAAAGTGCTGGGATTATAGGCATGAGCTACTGCACCC...,0.0
3,CHRX_22042679_22042879,TGGGAGCTGTATCAATCATGTTTTTTATTTTCTATATTTTATGATG...,0.0
4,CHRX_23280479_23280679,TACAGCAAATAGCCTTGGCAGATACAGTGTTTCCCTCCAGAGCAAA...,0.0


## Combine the data frames to form a single dataset

In [10]:
all_data_df = pd.concat([enhancer_df, non_enhancer_df])
all_data_df.reset_index(drop=True, inplace=True)
all_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2968 entries, 0 to 2967
Data columns (total 3 columns):
id          2968 non-null object
sequence    2968 non-null object
enhancer    2968 non-null float64
dtypes: float64(1), object(2)
memory usage: 69.6+ KB


In [11]:
all_data_df.head()

,id,sequence,enhancer
0,CHRX_48897056_48897256,CACAATGTAGAAGCAGAGACACAGGAACCAGGCTTGGTGATGGCTC...,1.0
1,CHR12_6444339_6444539,GCCCTCACATTCCCTGGCCCATCCCCTCCACCTCAAAATTTACAAA...,1.0
2,CHR12_6444939_6445139,GAGCAGGAGGCCAGTCACCCTGAGTCAGCCACGGGGAGACGCTGCA...,1.0
3,CHR12_6445139_6445339,CCTCTGCTGAGAACAGGACTGGGGCTTCCAGGGCAACAGGAAGGGT...,1.0
4,CHR12_6445339_6445539,ACAGCCTTAAAGGGAGCTTTTCAGGGACCTCTGGCCAGTGGGGGAT...,1.0


All the sequences are of length 200 characters.

In [12]:
all_data_df['sequence'].map(lambda x: len(x)).value_counts()

200    2968
Name: sequence, dtype: int64

## Load the physiochemical property data

In [13]:
pcp_df = pd.read_csv(os.path.join('data', 'S2.csv'), index_col=0)
pcp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, AA to TT
Data columns (total 6 columns):
Rise     16 non-null float64
Roll     16 non-null float64
Shift    16 non-null float64
Slide    16 non-null float64
Tilt     16 non-null float64
Twist    16 non-null float64
dtypes: float64(6)
memory usage: 896.0+ bytes


In [14]:
pcp_df

,Rise,Roll,Shift,Slide,Tilt,Twist
AA,7.65,2.26,1.69,0.026,0.020,0.038
AC,8.93,3.03,1.32,0.036,0.023,0.038
AG,7.08,2.03,1.46,0.031,0.019,0.037
AT,9.07,3.83,1.03,0.033,0.022,0.036
CA,6.38,1.78,1.07,0.016,0.017,0.025
CC,8.04,1.65,1.43,0.026,0.019,0.042
CG,6.23,2.00,1.08,0.014,0.016,0.026
CT,7.08,2.03,1.46,0.031,0.019,0.037
GA,8.56,1.93,1.32,0.025,0.020,0.038
GC,9.53,2.61,1.20,0.025,0.026,0.036


Normalise the physiochemical property values

In [15]:
scaler = MinMaxScaler()
pcp_df.loc[:, :] = scaler.fit_transform(pcp_df.values)
pcp_df

,Rise,Roll,Shift,Slide,Tilt,Twist
AA,0.430303,0.403042,1.000000,0.545455,0.4,0.833333
AC,0.818182,0.695817,0.618557,1.000000,0.7,0.833333
AG,0.257576,0.315589,0.762887,0.772727,0.3,0.791667
AT,0.860606,1.000000,0.319588,0.863636,0.6,0.750000
CA,0.045455,0.220532,0.360825,0.090909,0.1,0.291667
CC,0.548485,0.171103,0.731959,0.545455,0.3,1.000000
CG,0.000000,0.304183,0.371134,0.000000,0.0,0.333333
CT,0.257576,0.315589,0.762887,0.772727,0.3,0.791667
GA,0.706061,0.277567,0.618557,0.500000,0.4,0.833333
GC,1.000000,0.536122,0.494845,0.500000,1.0,0.750000


Transform 1 sequence

In [16]:
all_data_df['sequence'][0]

'CACAATGTAGAAGCAGAGACACAGGAACCAGGCTTGGTGATGGCTCTCAGGGGTCACAGTCTGATGGGGGACACACTGGAGGTCAGTCTGGTGGGGGAGTTTTAGCCTTTGGTCCTTATGGTGAAGCCTAGATTTGAGCCTGTTCACATATTAAGTGGAGATGCTATTGTTCAGCTCTGCAAGGGGGGGTTTGTCCTATT'

In [17]:
a_seq = all_data_df['sequence'][0]
[ a_seq[i:i+2] for i in range(len(a_seq) - 1) ][:5]

['CA', 'AC', 'CA', 'AA', 'AT']

In [18]:
a_seq = all_data_df['sequence'][0]
list(map(lambda x: pcp_df['Rise'][x], [ a_seq[i:i+2] for i in range(len(a_seq) - 1) ][:5]))

[0.04545454545454519,
 0.8181818181818181,
 0.04545454545454519,
 0.4303030303030304,
 0.860606060606061]

## Prepare the sequence data for modelling

Initialise the keras tokenizer

In [19]:
tokenizer =  Tokenizer(num_words=4, lower=False, char_level=True)

In [20]:
tokenizer.fit_on_texts(all_data_df['sequence'][0])

In [21]:
tokenizer.word_index

{'G': 1, 'T': 2, 'A': 3, 'C': 4}

In [22]:
tokenizer.index_word

{1: 'G', 2: 'T', 3: 'A', 4: 'C'}

Test the tokenizer on the first sequence

In [23]:
all_data_df['sequence'][0][:10]

'CACAATGTAG'

In [24]:
tokenizer.texts_to_matrix(all_data_df['sequence'][0], mode='binary')[:10]

array([[0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]])

Create a transformation pipleline to prepare the training dataset for RNN.

In [25]:
# This class selects the desired attributes and drops the rest.
class DataFrameSelector(BaseEstimator, TransformerMixin):

    def __init__(self, attribute_names):
        self.attribute_names = attribute_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.attribute_names]

In [26]:
# This class converts a nucleotide base (A, C, G, T) to one-hot-encoding.
class one_hot_encoder(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.tokenizer = Tokenizer(num_words=4, lower=False, char_level=True)

    def fit(self, X, y=None):
        # Note that X is a data frame.
        # Fit the tokenizer on the 1st sequence in the dataset.
        self.tokenizer.fit_on_texts(X.iloc[0, 0])
        self.len_sequence = len(X.iloc[0, 0])
        return self

    def transform(self, X):
        # Note that X is a data frame.
        one_hot_X = X.iloc[:, 0].map(lambda x: tokenizer.texts_to_matrix(x, mode='binary')).values
        one_hot_X = np.concatenate(one_hot_X)
        one_hot_X = np.reshape(one_hot_X, (-1, self.len_sequence, 4))
        return one_hot_X

In [27]:
# This class converts a sequence of nucleotide bases (A, C, G, T) to a sequence of dinucleotides and then to a sequence of pysiochemical properties of each dinucleotide.
class pcp_encoder(BaseEstimator, TransformerMixin):

    def __init__(self, pcp_df):
        self.pcp_df = pcp_df

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Note that X is a data frame.
        dinuc_seq = X.iloc[:, 0].map(lambda x: [ x[i:i+2] for i in range(len(x) - 1) ])
        pcp_seq = dinuc_seq.map(lambda x: [ pcp_df[j][i] for i in x for j in pcp_df.columns.tolist() ])
        # Pad with -1 for last element of sequence; it does not have an associated di-nucleotide
        pcp_seq = pcp_seq.map(lambda x: np.array(x + [-1. for i in range(len(pcp_df.columns))]).reshape((len(X.iloc[0, 0]), len(pcp_df.columns)))).values
        # pandas values returns a 1-D array of objects; use numpy stack to reshape it to a multi-dimensional array
        return np.stack(pcp_seq)

In [28]:
# This class shapes a numpy array.
class Array_Shaper(BaseEstimator, TransformerMixin):
    
    def __init__(self, shape):
        self.shape = shape
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.reshape(self.shape)

Test the pysiochemical property encoder

In [29]:
encoder = pcp_encoder(pcp_df)
test = encoder.transform(all_data_df['sequence'][:2].to_frame())
test[0][:5]

array([[0.04545455, 0.22053232, 0.36082474, 0.09090909, 0.1       ,
        0.29166667],
       [0.81818182, 0.69581749, 0.6185567 , 1.        , 0.7       ,
        0.83333333],
       [0.04545455, 0.22053232, 0.36082474, 0.09090909, 0.1       ,
        0.29166667],
       [0.43030303, 0.40304183, 1.        , 0.54545455, 0.4       ,
        0.83333333],
       [0.86060606, 1.        , 0.31958763, 0.86363636, 0.6       ,
        0.75      ]])

In [30]:
test[0][-5:]

array([[ 0.25757576,  0.31558935,  0.7628866 ,  0.77272727,  0.3       ,
         0.79166667],
       [ 0.        ,  0.        ,  0.        ,  0.13636364,  0.        ,
         0.        ],
       [ 0.86060606,  1.        ,  0.31958763,  0.86363636,  0.6       ,
         0.75      ],
       [ 0.43030303,  0.40304183,  1.        ,  0.54545455,  0.4       ,
         0.83333333],
       [-1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        ]])

In [31]:
test.shape

(2, 200, 6)

The FeatureUnion class in sklearn works only with 2D numpy arrays.

The input data is 1D (samples,) when one-hot encoded becomes 3D (samples, sequence length, number of bases). Similarly, after encoding using the dinucleotide pysiochemical properties, the output array is 3D. We will reshape both encoder outputs to 2D (samples * sequence length, number of bases) to let FeatureUnion combine them.

The final step in the transformation pipeline will reshape the combined arrays back to 3D.

In [32]:
attrbs = ['sequence']
num_bases = 4  # number of nucleotide bases
num_pcp = 6  # number of di-nucleotide physiochemical properties
len_seq = len(all_data_df['sequence'][0])
one_hot_pipeline = Pipeline([
    ('selector', DataFrameSelector(attrbs)),
    ('one_hot_encoder', one_hot_encoder()),
    ('array_shaper2D', Array_Shaper((-1, num_bases)))
])
pcp_pipeline = Pipeline([
    ('selector', DataFrameSelector(attrbs)),
    ('pcp_encoder', pcp_encoder(pcp_df)),
    ('array_shaper2D', Array_Shaper((-1, num_pcp)))
])
union_pipeline = FeatureUnion(transformer_list=[
    ("one_hot_pipeline", one_hot_pipeline),
    ("pcp_pipeline", pcp_pipeline)
])
my_pipeline = Pipeline([
    ('feature_combiner', union_pipeline),
    ('array_shaper3D', Array_Shaper((-1, len_seq, num_bases + num_pcp)))
])

The final output shape below is correct - 10 features for each nucleotide base (4 from one-hot encoding and 6 physiochemical properties), 200 bases in each sequence and 2968 samples.

In [33]:
X = my_pipeline.fit_transform(all_data_df)
X.shape

(2968, 200, 10)

Check the 1st sequence is correctly encoded.

In [34]:
X[0, :10, :]

array([[0.        , 0.        , 0.        , 0.        , 0.04545455,
        0.22053232, 0.36082474, 0.09090909, 0.1       , 0.29166667],
       [0.        , 0.        , 0.        , 1.        , 0.81818182,
        0.69581749, 0.6185567 , 1.        , 0.7       , 0.83333333],
       [0.        , 0.        , 0.        , 0.        , 0.04545455,
        0.22053232, 0.36082474, 0.09090909, 0.1       , 0.29166667],
       [0.        , 0.        , 0.        , 1.        , 0.43030303,
        0.40304183, 1.        , 0.54545455, 0.4       , 0.83333333],
       [0.        , 0.        , 0.        , 1.        , 0.86060606,
        1.        , 0.31958763, 0.86363636, 0.6       , 0.75      ],
       [0.        , 0.        , 1.        , 0.        , 0.04545455,
        0.22053232, 0.36082474, 0.09090909, 0.1       , 0.29166667],
       [0.        , 1.        , 0.        , 0.        , 0.81818182,
        0.69581749, 0.6185567 , 1.        , 0.7       , 0.83333333],
       [0.        , 0.        , 1.       

In [35]:
X[0, -10:, :]

array([[ 0.        ,  0.        ,  1.        ,  0.        ,  0.43030303,
         0.40304183,  1.        ,  0.54545455,  0.4       ,  0.83333333],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.04545455,
         0.22053232,  0.36082474,  0.09090909,  0.1       ,  0.29166667],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.81818182,
         0.69581749,  0.6185567 ,  1.        ,  0.7       ,  0.83333333],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.70606061,
         0.27756654,  0.6185567 ,  0.5       ,  0.4       ,  0.83333333],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.54848485,
         0.17110266,  0.73195876,  0.54545455,  0.3       ,  1.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.25757576,
         0.31558935,  0.7628866 ,  0.77272727,  0.3       ,  0.79166667],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.13636364

In [36]:
y = all_data_df['enhancer'].values
y.shape

(2968,)

In [37]:
y[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Save the transformed data

In [38]:
np.save(os.path.join('data', 'train_X.npy'), X)
np.save(os.path.join('data', 'train_y.npy'), y)